The first cell is used to import the packages needed to scrape the table from the website.
Rows are indicated by a 'tr' so finding that marker will allow the rows of the data frame to be populated.
Print the datafram to ensure all elements are there.  

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table')
rows = table.find_all('tr')
data = []
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
df = pd.DataFrame(data, columns=['Postcode', 'Borough', 'Neighborhood'])


Takes the Borough with a 'Not assigned' and removes the row from the data frame. 
Shape call show the number of row and the number of columns.

In [2]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.shape

(211, 3)

Reads in the second data frame with the postal code and coordinates. 

In [3]:
data = pd.read_csv('https://cocl.us/Geospatial_data')
df2 = data
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Renaming the Postal Code column in order for the two data frames to be merged easily.

In [4]:
df2.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df2.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two data frames so the neighborhoods will have coordinates. 

In [5]:
df3 = pd.merge(df, df2, on=['Postcode'], how='inner')
df3.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


Import the packages needed to make a folium map and run the Kmeans clustering.

In [6]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



Foursquare credentials and obtaining venue information from Foursquare based on the first Toronto neighborhood.  

In [8]:
CLIENT_ID = 'HYTIGFNWQZAGVBXYF1SY5DTO2VPKGGUEYM0HCIL1Z3F2Y1Q2' 
CLIENT_SECRET = 'PAMCF4KBKXDTIS5V2TBQYZCQXGV0RSSWWYXEPCKMPWKAOHJM' 
VERSION = '20180605' 

In [9]:
latitude=df3.loc[0, 'Latitude']
longitude=df3.loc[0, 'Longitude']
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude,
    longitude, 
    radius, 
    LIMIT)

'https://api.foursquare.com/v2/venues/explore?&client_id=HYTIGFNWQZAGVBXYF1SY5DTO2VPKGGUEYM0HCIL1Z3F2Y1Q2&client_secret=PAMCF4KBKXDTIS5V2TBQYZCQXGV0RSSWWYXEPCKMPWKAOHJM&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

Store the results of the call and create a function to search the first neighborhood for venues.  

In [10]:
results = requests.get(url).json()

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Store the results for each neighborhood and clean the data for easier visualization. 

In [12]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


Define a function to iterate through the dataframe to find all the venues in the neighborhoods.

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=df3['Neighborhood'], latitudes=df3['Latitude'],longitudes=df3['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Not assigned
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

Group the venues by the neighborhoods.

In [15]:
toronto_venues.groupby('Neighborhood').count()

(4389, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Albion Gardens,7,7,7,7,7,7
Alderwood,9,9,9,9,9,9
Bathurst Manor,19,19,19,19,19,19
Bathurst Quay,15,15,15,15,15,15
Bayview Village,4,4,4,4,4,4
Beaumond Heights,7,7,7,7,7,7


Add columns for venue types and sort the dataframe for ease of use. 

In [16]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the venue data based on the mean in each neighborhood. 

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Agincourt North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Albion Gardens,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Alderwood,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,Bathurst Manor,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0
6,Bathurst Quay,0.000000,0.0,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.066667,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Beaumond Heights,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,Bedford Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


Define a function to find the most common venues and run for the top 10 venues.

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Cosmetics Shop,Steakhouse,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Restaurant,American Restaurant
1,Agincourt,Clothing Store,Lounge,Breakfast Spot,Latin American Restaurant,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,Agincourt North,Park,Playground,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,Albion Gardens,Pharmacy,Beer Store,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Grocery Store,Pizza Place,Event Space,Ethiopian Restaurant,Empanada Restaurant
4,Alderwood,Pizza Place,Pharmacy,Skating Rink,Coffee Shop,Pool,Pub,Sandwich Place,Gym,Airport Service,Field


run the kmeans clustering algorithm on the dataframe to group the data into 5 clusters. Add the clusters to the dataframe for catagorization. 

In [22]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([4, 4, 1, 2, 4, 4, 4, 4, 2, 4], dtype=int32)

In [23]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df3
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Portuguese Restaurant,Hockey Arena,Coffee Shop,Intersection,Pizza Place,Financial or Legal Service,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4.0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Mexican Restaurant,Dessert Shop,Shoe Store
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4.0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Mexican Restaurant,Dessert Shop,Shoe Store
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,4.0,Clothing Store,Women's Store,Boutique,Fraternity House,Athletics & Sports,Miscellaneous Shop,Arts & Crafts Store,Coffee Shop,Event Space,Vietnamese Restaurant


Remove all the 'NaN' valuses from the Cluster Labels column.  

In [45]:
a = toronto_merged.dropna(subset=['Cluster Labels'])

tt = pd.DataFrame(a)

0      1.0
1      4.0
2      4.0
3      4.0
4      4.0
5      4.0
6      4.0
8      2.0
9      2.0
10     4.0
11     2.0
12     2.0
13     4.0
14     4.0
15     4.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     2.0
22     2.0
23     2.0
24     4.0
25     4.0
26     4.0
27     4.0
28     4.0
29     4.0
30     4.0
      ... 
180    2.0
181    2.0
182    2.0
183    1.0
184    4.0
185    4.0
186    4.0
187    2.0
189    4.0
190    4.0
191    4.0
192    4.0
193    1.0
194    1.0
195    1.0
196    4.0
197    4.0
198    3.0
199    3.0
200    3.0
201    3.0
202    3.0
203    3.0
204    3.0
205    3.0
206    4.0
207    4.0
208    4.0
209    4.0
210    4.0
Name: Cluster Labels, Length: 209, dtype: float64

Create the map of Toronto with the clusters in different colors.  

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(tt['Latitude'], tt['Longitude'], tt['Neighborhood'], tt['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters